In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, make_scorer, auc, roc_curve

# Import data
bank = pd.read_csv('bank-additional-full.csv', sep=';')

In [3]:
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
# Define functions for data preprocessing.

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def One_hot_encoding(dataset, attr):
    
    ''' Perform One Hot Encoding on an attribute of a dataset.
    Arguments:
        -- dataset: the data set.
        -- attr: the attribute name, in string form.
        
    Output:
        return the dataset with added encoded attributes. And the original attribute is dropped'''
        
    ohc = OneHotEncoder()
    ohe = ohc.fit_transform(dataset[attr].values.reshape(-1,1)).toarray()
    dfOneHot = pd.DataFrame(ohe,columns = [attr + '_' + str(ohc.categories_[0][i]) for i in range(len(ohc.categories_[0]))])
    dataset = pd.concat([dataset,dfOneHot], axis=1)
    dataset.drop([attr], axis = 1,inplace=True)
    
    return dataset

def Ordinal_encoding(dataset, attr, category = 'auto'):
    
    ''' Perform One Hot Encoding on an attribute of a dataset.
    Arguments:
        -- dataset: the data set.
        -- attr: the attribute name, in string form.
        -- category: should be a list, with the ith element is encoded as i (integer). default: 'auto'.
        
    Output:
        return the dataset with added encoded attribute. And the original attribute is dropped'''
    enc = OrdinalEncoder(categories = [category])
    dataset[attr + '_encoded'] = enc.fit_transform(dataset[attr].values.reshape(-1,1))
    dataset.drop([attr], axis = 1,inplace=True)
    
    return dataset

def label_encoding(dataset, attr):
    
    ''' Perform Label Encoding on the target attribute of a dataset.
    Arguments:
        -- dataset: the data set.
        -- attr: the target attribute, in string form.
        
    Output:
        return the dataset with encoded target attribute, with the original attribute dropped.'''
        
    le = LabelEncoder()
    dataset[attr + '_encoded'] = le.fit_transform(dataset[attr])
    dataset.drop([attr], axis = 1,inplace=True)
    
    return dataset    

def standardization(dataset, attr):
    
    ''' Scale the numeric attribute of the dataset to zero mean unit variance.
    Arguments:
        -- dataset: the data set.
        -- attr: the attribute name, in string form.
        
    Output:
        return the dataset with standardized attribute. And the original attribute is dropped'''
        
    ss = StandardScaler()
    dataset[attr +'_scaled'] = ss.fit_transform(dataset[attr].values.reshape(-1,1))
    dataset.drop([attr], axis = 1,inplace=True)
    
    return dataset    

Data preprocessing.

1. The target attribute has two values. Encode "no" and "yes" to 0 and 1, respectively.

2. The column "pdays" has too many missing values and is not useable. Drop it.

3. The column "education" is an ordinal attribute. Convert it into integers, with higher values representing higher levels of education.

4. Encode other categorical variables using one-hot encoding and scale to zero mean unit variance.

In [3]:
# encode label
bank = label_encoding(bank, 'y')  

# drop unwanted columns.
bank.drop(['pdays'], axis = 1, inplace=True)

# preprocess ordinal variable: 'education'
bank = Ordinal_encoding(bank, 'education', ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'unknown', 'high.school','professional.course', 'university.degree'])

# encode nominal categorical variables ('job', 'marital',  'default', 'housing', 'loan', 'contact','poutcome')
bank = One_hot_encoding(bank, 'job')
bank = One_hot_encoding(bank, 'marital')
bank = One_hot_encoding(bank, 'default')
bank = One_hot_encoding(bank, 'housing')
bank = One_hot_encoding(bank, 'contact')
bank = One_hot_encoding(bank, 'poutcome')
bank = One_hot_encoding(bank, 'loan')

# encode time variables ('month', 'day_of_week') using One-Hot encoder  
bank = One_hot_encoding(bank, 'month')
bank = One_hot_encoding(bank, 'day_of_week')

# standardize the numerical variables ('duration', 'campaign', 'pdays','previous','emp.var.rate',
# 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed')
bank = standardization(bank, 'age')
bank = standardization(bank, 'duration')
bank = standardization(bank, 'campaign')
bank = standardization(bank, 'previous')
bank = standardization(bank, 'emp.var.rate')
bank = standardization(bank, 'cons.price.idx')
bank = standardization(bank, 'cons.conf.idx')
bank = standardization(bank, 'euribor3m')
bank = standardization(bank, 'nr.employed')
bank = standardization(bank, 'education_encoded')

In [4]:
# Split data set. Test set: 20%. Validation set + training set: 80%

from sklearn.model_selection import train_test_split

y = bank.iloc[:,bank.columns == 'y_encoded'].values.ravel() # convert to 1D, to meet the requirement of fit_transform()
X = bank.iloc[:, bank.columns != 'y_encoded']
        
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =.20, random_state = 1, stratify = y)

In [5]:
# Experiments show that the linear kernel performs better than any other kernels. 
# In this part, fine tune the hyperparameters of the classifier with a linear kernel.

from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

classifier = SVC(kernel = 'linear', class_weight = 'balanced')
distributions = dict(C = uniform(loc = 1, scale = 10))
scoring = {
    'recall': make_scorer(recall_score),
    'roc_auc': make_scorer(roc_auc_score)
}
nfolds = 3
n_iter = 15

rs = RandomizedSearchCV(estimator = classifier, param_distributions = distributions, scoring = scoring, n_iter = n_iter, 
                        refit = 'roc_auc', cv = nfolds)
search = rs.fit(X_train, y_train)
results = pd.DataFrame(search.cv_results_)

{'C': 3.7826705684779505}


In [7]:
# Evaluation result of the best model

best_model = SVC(C = 6.507979025745755, kernel = 'linear', class_weight = 'balanced')
best_model.fit(X_train,y_train)
y_pred = best_model.predict(X_test)
y_score = best_model.decision_function(X_test)

recall = recall_score(y_test, y_pred) 
roc_auc = roc_auc_score(y_test, y_score) 
cm = confusion_matrix(y_test, y_pred) 
print('roc_auc: ', roc_auc)
print('recall: ', recall)
print('confusion_matrix: ')
print(cm)

roc_auc:  0.9328619569319307
recall:  0.8890086206896551
confusion_matrix: 
[[6175 1135]
 [ 103  825]]


SVM has good performance but is computationally expensive.